In [1]:
# put this in it's own codeblock because it'll keep 
# running otherwise and eventually move to the root dir lol
%cd ../..

/Users/angus/code/jupyter/notebooks/20240907, vg


/Users/angus/code/jupyter/env/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
from lib.service.database import DatabaseService
from lib.defaults import INSTANCE_CFG
import os

os.makedirs('_out_img', exist_ok=True)

db = DatabaseService.create(INSTANCE_CFG[3].database, 1)
await db.wait_till_running()

dbname=au_land_db_3 port=5431 user=postgres host=localhost password=throwAwayPassword3


In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.patches import Patch


df = gpd.read_postgis("""
select l.*, ROUND(v.area_variance::numeric, 2) as area_variance
  from (
    select lot_id_string, (MAX(shape_area) - MIN(shape_area)) AS area_variance
      from nsw_spatial_lppt_raw.lot_feature_layer
      group by lot_id_string
      order by 2 DESC) v
  left join nsw_spatial_lppt_raw.lot_feature_layer l using (lot_id_string)
  where area_variance > 0
  order by area_variance DESC, l.last_update DESC
""", db.engine(), geom_col='geometry')

lot_id_strings = df['lot_id_string'].unique().tolist()
resolution = 8
plot_cols, plot_rows = 12, 9

for offset in range(0, len(lot_id_strings), plot_cols * plot_rows):
    figsize = (plot_cols * resolution, plot_rows * resolution)
    fig, ax = plt.subplots(plot_cols, plot_rows, figsize=figsize)
    for subplot in range(0, min(plot_cols * plot_rows, len(lot_id_strings) - offset)):
        ax_col = subplot % plot_cols
        ax_row = subplot // plot_rows
        sub_ax = ax[ax_row][ax_col]
        lot_id = lot_id_strings[offset + subplot]
        sub_df = df[df['lot_id_string'] == lot_id].reset_index(drop=True)

        sub_df.iloc[[1]].plot(ax=sub_ax, color='none', edgecolor='blue', hatch='\\\\\\')
        sub_df.iloc[[0]].plot(ax=sub_ax, color='none', edgecolor='red', hatch='///')
        sub_ax.set_title(lot_id)
        sub_ax.set_xlabel(f"Variance {sub_df.loc[0, 'area_variance']}", loc='left')
        sub_ax.legend(handles=[
            Patch(facecolor='white', edgecolor='red', label=sub_df.loc[0, 'start_date']),
            Patch(facecolor='white', edgecolor='blue', label=sub_df.loc[1, 'start_date']),
        ], loc='lower right')
    plt.show()
